In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np

*ENABLE TPU *

In [3]:
train_data=pd.read_csv('fashion-mnist_train.csv')

In [4]:
x_train=train_data.iloc[:,1:].values
x_train=x_train/255
y_train=train_data.iloc[:,0:1].values

In [6]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)

# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

# instantiating the model in the strategy scope creates the model on the TPU
with tpu_strategy.scope():
  cnn_model=tf.keras.models.Sequential()
  cnn_model.add(tf.keras.layers.Dense(units=2048,activation='relu'))
  cnn_model.add(tf.keras.layers.Dropout(0.1))
  cnn_model.add(tf.keras.layers.Dense(units=2048,activation='relu'))
  cnn_model.add(tf.keras.layers.Dropout(0.1))
  cnn_model.add(tf.keras.layers.Dense(units=2048,activation='relu'))
  cnn_model.add(tf.keras.layers.Dropout(0.1))
  cnn_model.add(tf.keras.layers.Dense(units=2048,activation='relu'))
  cnn_model.add(tf.keras.layers.Dropout(0.15))
  cnn_model.add(tf.keras.layers.Dense(units=2048,activation='relu'))
  cnn_model.add(tf.keras.layers.Dropout(0.15))
  cnn_model.add(tf.keras.layers.Dense(units=1024,activation='relu'))
  cnn_model.add(tf.keras.layers.Dropout(0.05))
  cnn_model.add(tf.keras.layers.Dense(units=1024,activation='relu'))
  cnn_model.add(tf.keras.layers.Dropout(0.05))
  cnn_model.add(tf.keras.layers.Dense(units=10,activation='softmax'))
  cnn_model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['sparse_categorical_accuracy'])

cnn_model.fit(x_train,y_train,epochs=30,batch_size=64) 


938/938 [==============================] - 24s 26ms/step - loss: 0.3338 - sparse_categorical_accuracy: 0.8822
Epoch 10/30
938/938 [==============================] - 26s 27ms/step - loss: 0.3382 - sparse_categorical_accuracy: 0.8825
Epoch 11/30
938/938 [==============================] - 25s 27ms/step - loss: 0.3268 - sparse_categorical_accuracy: 0.8841
Epoch 12/30
938/938 [==============================] - 25s 26ms/step - loss: 0.3151 - sparse_categorical_accuracy: 0.8876
Epoch 13/30
938/938 [==============================] - 24s 26ms/step - loss: 0.3142 - sparse_categorical_accuracy: 0.8892
Epoch 14/30
938/938 [==============================] - 24s 26ms/step - loss: 0.2985 - sparse_categorical_accuracy: 0.8942
Epoch 15/30
938/938 [==============================] - 25s 26ms/step - loss: 0.3001 - sparse_categorical_accuracy: 0.8934
Epoch 16/30
938/938 [==============================] - 24s 26ms/step - loss: 0.2944 - sparse_categorical_accuracy: 0.8956
Epoch 17/30
938/938 [===============

In [7]:
test_data=pd.read_csv('fashion-mnist_test.csv')
x_test=test_data.iloc[:,1:].values
y_test=test_data.iloc[:,0:1].values

from sklearn.metrics import confusion_matrix,accuracy_score

In [8]:
y_pred=cnn_model.predict(x_test)

In [9]:
for i in range(y_pred.shape[0]):
    res=y_pred[i].max(axis=0)
    for j in range(y_pred.shape[1]):
        if y_pred[i,j]==res:
            y_pred[i,j]=j
        else:
            y_pred[i,j]=0

pred=np.zeros((y_pred.shape[0], 1))

for m in range (y_pred.shape[0]):
    for n in range(y_pred.shape[1]):
        if y_pred[m,n]==n:
            pred[m]=n
        else:
            pass

pred=pred.astype(np.int)

In [12]:
pred

array([[0],
       [1],
       [2],
       ...,
       [8],
       [8],
       [1]])

In [13]:
y_test

array([[0],
       [1],
       [2],
       ...,
       [8],
       [8],
       [1]])

In [20]:
def accuracy(test,hat):
  acc=np.zeros((test.shape[0],1))
  for i in range(test.shape[0]):
    if test[i]==hat[i]:
      acc[i]=1
    else:
      pass
  score=np.sum(acc)
  accuracy=(score/test.shape[0])*100
  print('{} %'.format(accuracy))

In [21]:
accuracy(y_test,pred)

85.69 %
